In [1]:
import os
# Root directory of the project
ROOT_DIR = os.path.abspath("../../../")
print(ROOT_DIR)

/Users/wubai.zhou/Uber/kaggle


In [3]:
import sys
# add src directory to system path
sys.path.append(os.path.join(ROOT_DIR, 'src'))

In [8]:
%load_ext autoreload
%autoreload 2

In [32]:
from relevance.config import config

In [33]:
super(type(config))

<super: relevance.config.ParamConfig, None>

In [34]:
config.__dict__

{'basic_tfidf_ngram_range': (1, 3),
 'basic_tfidf_vocabulary_type': 'common',
 'cooccurrence_tfidf_ngram_range': (1, 1),
 'cooccurrence_word_exclude_stopword': False,
 'count_feat_transform': <ufunc 'sqrt'>,
 'data_folder': '/Users/wubai.zhou/Uber/kaggle/data/relevance',
 'drop_html_flag': True,
 'feat_folder': '/Users/wubai.zhou/Uber/kaggle/data/relevance/Feat/solution',
 'n_classes': 4,
 'n_folds': 3,
 'n_runs': 3,
 'original_test_data_path': '/Users/wubai.zhou/Uber/kaggle/data/relevance/test.csv',
 'original_train_data_path': '/Users/wubai.zhou/Uber/kaggle/data/relevance/train.csv',
 'pos_tagged_test_data_path': '/Users/wubai.zhou/Uber/kaggle/data/relevance/Feat/solution/test.pos_tagged.csv.pkl',
 'pos_tagged_train_data_path': '/Users/wubai.zhou/Uber/kaggle/data/relevance/Feat/solution/train.pos_tagged.csv.pkl',
 'processed_test_data_path': '/Users/wubai.zhou/Uber/kaggle/data/relevance/Feat/solution/test.processed.csv.pkl',
 'processed_train_data_path': '/Users/wubai.zhou/Uber/kaggl

In [39]:
import _pickle as cPickle
import numpy as np
import pandas as pd
from relevance.utils.nlp_utils import clean_text
from relevance.config import config

In [75]:
# load data
print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path).fillna("")
dfTest = pd.read_csv(config.original_test_data_path).fillna("")
# number of train/test samples
num_train, num_test = dfTrain.shape[0], dfTest.shape[0]

print("Done.")

Load data...
Done.


In [76]:
dfTrain

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471
5,8,oakley polarized radar,Oakley Sunglasses - Radar Path Polished Black/...,"Durability and all-day comfort of lightweight,...",3,1.166
6,10,boyfriend jeans,How To Make An American Quilt (DVD),ITEM#: 2518897\nDirector Jocelyn Moorhouse's f...,1,0.000
7,13,screen protector samsung,ZAGG InvisibleShield Cell Phone Screen Protect...,Protect your most precious technology with the...,4,0.000
8,17,pots and pans set,Cook N Home Stainless Steel 4-Piece Pasta Cook...,This ultimate pasta and vegetable steamer cook...,2,0.632
9,20,waffle maker,Presto FlipSide Electric Waffle Maker- 03510,Please the waffle lover in your family with th...,4,0.471


In [77]:
dfTest

,id,query,product_title,product_description
0,3,electric griddle,Star-Max 48 in Electric Griddle,
1,6,phillips coffee maker,Philips SENSEO HD7810 WHITE Single Serve Pod C...,
2,9,san francisco 49ers,2013 San Francisco 49ers Clock,A 2013 San Francisco 49ers clock is the ultima...
3,11,aveeno shampoo,AVEENO 10.5FLOZ NRSH SHINE SH,"Water, Ammonium Lauryl Sulfate, Dimethicone, S..."
4,12,flea and tick control for dogs,Merial Frontline Plus Flea and Tick Control fo...,
5,14,table clock,Classy Wood Table Clock,Watch out for this antique wood table clock wh...
6,15,infinity scarf,Women's Infinity Scarf - Tribal Paisley,The style possibilities are almost endless wit...
7,16,workout clothes for women,Women's Skechers Flex Tech Tee Shirt White,Flex your style in the SKECHERS Apparel Flex T...
8,18,lego star wars,LEGO Star Wars Anakin's Jedi Interceptor 9494 ...,<ul>\n\t\t<li>\n\t\t\tEnglish \n\t\t\t\t</li>\...
9,19,kitchenaid mixer,Hamilton Beach Power Deluxe 4-Quart Stand Mixer,Get the mixing action you've always wanted wit...


In [78]:
# Preprocess data
print("Pre-process data...")

# insert fake label for test
dfTest["median_relevance"] = np.ones((num_test))
dfTest["relevance_variance"] = np.zeros((num_test))

# insert sample index
dfTrain["index"] = np.arange(num_train)
dfTest["index"] = np.arange(num_test)

# one-hot encode the median_relevance
for i in range(config.n_classes):
    dfTrain["median_relevance_%d" % (i + 1)] = 0
    dfTrain["median_relevance_%d" % (i + 1)][dfTrain["median_relevance"] == (i + 1)] = 1

# query ids
qid_dict = dict()
for i, q in enumerate(np.unique(dfTrain["query"]), start=1):
    qid_dict[q] = i

# insert query id
dfTrain["qid"] = list(map(lambda q: qid_dict[q], dfTrain["query"]))
dfTest["qid"] = list(map(lambda q: qid_dict[q], dfTest["query"]))

# clean text
clean = lambda line: clean_text(line, drop_html_flag=config.drop_html_flag)
dfTrain = dfTrain.apply(clean, axis=1)
dfTest = dfTest.apply(clean, axis=1)

print("Done.")

Pre-process data...


/Users/wubai.zhou/Uber/kaggle/kenv/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/wubai.zhou/Uber/kaggle/kenv/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65497012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/wubai.zhou/Uber/kaggle/kenv/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65516012.jpg" looks like a URL. Beauti

Done.


In [79]:
dfTrain

,id,query,product_title,product_description,median_relevance,relevance_variance,index,median_relevance_1,median_relevance_2,median_relevance_3,median_relevance_4,qid
0,1,bridal shower decorations,accent pillow with heart design - red/black,red satin accent pillow embroidered with a hea...,1,0.000,0,1,0,0,0,31
1,2,led christmas lights,set of 10 battery operated multi led train chr...,set of 10 battery operated train christmas lig...,4,0.000,1,0,0,0,1,122
2,4,projector,viewsonic pro8200 dlp multimedia projector,,4,0.471,2,0,0,0,1,171
3,5,wine rack,concept housewares wr-44526 solid-wood ceiling...,"like a silent and sturdy tree, the southern en...",4,0.000,3,0,0,0,1,250
4,7,light bulb,wintergreen lighting christmas led light bulb ...,"wtgr1011\nfeatures\nnickel base, 60,000 averag...",2,0.471,4,0,1,0,0,128
5,8,oakley polarized radar,oakley sunglasses - radar path polished black/...,"durability and all-day comfort of lightweight,...",3,1.166,5,0,0,1,0,152
6,10,boyfriend jeans,how to make an american quilt (dvd),item#: 2518897\ndirector jocelyn moorhouse's f...,1,0.000,6,1,0,0,0,29
7,13,screen protector samsung,zagg invisibleshield cell phone screen protect...,protect your most precious technology with the...,4,0.000,7,0,0,0,1,193
8,17,pots and pans set,cook n home stainless steel 4-piece pasta cook...,this ultimate pasta and vegetable steamer cook...,2,0.632,8,0,1,0,0,168
9,20,waffle maker,presto flipside electric waffle maker- 03510,please the waffle lover in your family with th...,4,0.471,9,0,0,0,1,239


In [80]:
dfTest

,id,query,product_title,product_description,median_relevance,relevance_variance,index,qid
0,3,electric griddle,star-max 48 in electric griddle,,1.0,0.0,0,72
1,6,phillips coffee maker,philips senseo hd7810 white single serve pod c...,,1.0,0.0,1,159
2,9,san francisco 49ers,2013 san francisco 49ers clock,a 2013 san francisco 49ers clock is the ultima...,1.0,0.0,2,192
3,11,aveeno shampoo,aveeno 10.5floz nrsh shine sh,"water, ammonium lauryl sulfate, dimethicone, s...",1.0,0.0,3,12
4,12,flea and tick control for dogs,merial frontline plus flea and tick control fo...,,1.0,0.0,4,79
5,14,table clock,classy wood table clock,watch out for this antique wood table clock wh...,1.0,0.0,5,220
6,15,infinity scarf,woman infinity scarf - tribal paisley,the style possibilities are almost endless wit...,1.0,0.0,6,106
7,16,workout clothes for woman,woman skechers flex tech tee shirt white,flex your style in the skechers apparel flex t...,1.0,0.0,7,253
8,18,lego star wars,lego star wars anakin's jedi interceptor 9494 ...,\n\t\t \n\t\t\tenglish \n\t\t\t\t \n \t \n ...,1.0,0.0,8,124
9,19,kitchenaid mixer,hamilton beach power deluxe 4-quart stand mixer,get the mixing action you've always wanted wit...,1.0,0.0,9,116


In [82]:
# save data
print("Save data...")

with open(config.processed_train_data_path, "wb") as f:
    cPickle.dump(dfTrain, f, -1)
with open(config.processed_test_data_path, "wb") as f:
    cPickle.dump(dfTest, f, -1)

print("Done.")

Save data...
Done.


In [121]:
# load data
with open(config.processed_train_data_path, "rb") as f:
    dfTrain = cPickle.load(f)

skf = [0] * config.n_runs
for stratified_label, key in zip(["relevance", "query"],
                                 ["median_relevance", "qid"]):
    for run in range(config.n_runs):
        random_seed = 2018 + 1000 * (run + 1)
        skf[run] = StratifiedKFold(dfTrain[key],
                                   n_folds=config.n_folds,
                                   shuffle=True,
                                   random_state=random_seed)
        for fold, (validInd, trainInd) in enumerate(skf[run]):
            print("================================")
            print("Index for run: %s, fold: %s" % (run + 1, fold + 1))
            print("Train (num = %s)" % len(trainInd))
            print(trainInd[:10])
            print("Valid (num = %s)" % len(validInd))
            print(validInd[:10])
    with open("%s/stratifiedKFold.%s.pkl" %
              (config.data_folder, stratified_label), "wb") as f:
        cPickle.dump(skf, f, -1)

Index for run: 1, fold: 1
Train (num = 3386)
[ 3  4  9 15 26 28 29 34 36 37]
Valid (num = 6772)
[ 0  1  2  5  6  7  8 10 11 12]
Index for run: 1, fold: 2
Train (num = 3386)
[ 0  2  6  7 11 12 13 14 16 19]
Valid (num = 6772)
[ 1  3  4  5  8  9 10 15 17 18]
Index for run: 1, fold: 3
Train (num = 3386)
[ 1  5  8 10 17 18 20 24 31 40]
Valid (num = 6772)
[ 0  2  3  4  6  7  9 11 12 13]
Index for run: 2, fold: 1
Train (num = 3386)
[ 1  6  8 12 14 15 18 26 31 35]
Valid (num = 6772)
[ 0  2  3  4  5  7  9 10 11 13]
Index for run: 2, fold: 2
Train (num = 3386)
[ 0  3  4  9 10 11 13 16 20 21]
Valid (num = 6772)
[ 1  2  5  6  7  8 12 14 15 17]
Index for run: 2, fold: 3
Train (num = 3386)
[ 2  5  7 17 19 25 27 28 29 30]
Valid (num = 6772)
[ 0  1  3  4  6  8  9 10 11 12]
Index for run: 3, fold: 1
Train (num = 3386)
[ 2  3  9 10 12 13 14 16 17 21]
Valid (num = 6772)
[ 0  1  4  5  6  7  8 11 15 18]
Index for run: 3, fold: 2
Train (num = 3386)
[ 4  8 15 19 20 23 27 30 42 49]
Valid (num = 6772)
[ 0  1  

In [83]:
from nltk.corpus import wordnet

In [87]:
word = "stop"
syns = wordnet.synsets(word, pos=None)
syns

[Synset('stop.n.01'),
 Synset('stop.n.02'),
 Synset('stop.n.03'),
 Synset('arrest.n.02'),
 Synset('stop.n.05'),
 Synset('stop_consonant.n.01'),
 Synset('period.n.07'),
 Synset('stop.n.08'),
 Synset('diaphragm.n.01'),
 Synset('catch.n.06'),
 Synset('blockage.n.02'),
 Synset('stop.v.01'),
 Synset('discontinue.v.01'),
 Synset('stop.v.03'),
 Synset('stop.v.04'),
 Synset('stop.v.05'),
 Synset('break.v.10'),
 Synset('check.v.18'),
 Synset('intercept.v.01'),
 Synset('end.v.01'),
 Synset('barricade.v.01'),
 Synset('hold_on.v.02')]

In [94]:
lemmas = syns[3].lemmas()
lemmas

[Lemma('arrest.n.02.arrest'),
 Lemma('arrest.n.02.check'),
 Lemma('arrest.n.02.halt'),
 Lemma('arrest.n.02.hitch'),
 Lemma('arrest.n.02.stay'),
 Lemma('arrest.n.02.stop'),
 Lemma('arrest.n.02.stoppage')]

In [109]:
antonyms = set()
for syn in wordnet.synsets("stop", pos=None):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())

In [111]:
antonyms

{'begin', 'continuant_consonant', 'continue', 'start'}

In [113]:
list(zip(["relevance", "query"], ["median_relevance", "qid"]))

[('relevance', 'median_relevance'), ('query', 'qid')]

In [116]:
from sklearn.cross_validation import StratifiedKFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1, 0, 1])
skf = StratifiedKFold(y, n_folds=3)

In [118]:
skf

sklearn.cross_validation.StratifiedKFold(labels=[0 0 1 1 0 1], n_folds=3, shuffle=False, random_state=None)

In [119]:
for train_index, test_index in skf:
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [1 3 4 5] TEST: [0 2]
TRAIN: [0 2 4 5] TEST: [1 3]
TRAIN: [0 1 2 3] TEST: [4 5]


In [122]:
config.data_folder

'/Users/wubai.zhou/Uber/kaggle/data/relevance'

In [135]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})
df

,A,B,C,D
0,foo,one,0.492772,0.167057
1,bar,one,0.253750,0.607155
2,foo,two,0.704902,0.886208
3,bar,three,-0.619749,0.008464
4,foo,two,-1.501179,-1.366336
5,bar,two,0.575006,0.321888
6,foo,one,0.543782,-0.409687
7,foo,three,0.742578,-2.028067


In [146]:
df["sample_index"] = range(df.shape[0])
grouped = df.groupby(['A', 'B'])
grouped

In [150]:
agg = grouped.apply(lambda x: list(x["sample_index"]))
agg

A    B    
bar  one         [1]
     three       [3]
     two         [5]
foo  one      [0, 6]
     three       [7]
     two      [2, 4]
dtype: object

In [152]:
dict(agg)

{('bar', 'one'): [1],
 ('bar', 'three'): [3],
 ('bar', 'two'): [5],
 ('foo', 'one'): [0, 6],
 ('foo', 'three'): [7],
 ('foo', 'two'): [2, 4]}

In [153]:
np.arange(0, 1.5, 0.5)

array([0. , 0.5, 1. ])